In [1]:
import numpy as np
import os

In [2]:
def detect_namespace(namespace:str):
    content = namespace[namespace.find("(")+1:-1].split(", ")
    arg_dict = {}
    for arg in content:
        arg_type, arg_value = arg.split("=")
        arg_dict[arg_type] = arg_value
    return arg_dict

def get_acc(line:str, delimiter=":", precision=4):
    index = line.find(delimiter) + len(delimiter)
    acc = line[index:index + 3 + precision]
    return float(acc)

def parse_file(f):
    for line in f:
        if line.find("Namespace") != -1:
            arg_dict = detect_namespace(line)
        if line.find("No mask noise average acc") != -1:
            NM = get_acc(line)
        if line.find("Finetune noise average acc") != -1:
            FT = get_acc(line)
    return arg_dict, NM, FT

In [3]:
class PResultItem():
    def __init__(self, main_key:float):
        self.main_key = main_key
        self.NM_list = []
        self.FT_list = []
    
    def add_result(self, NM:float, FT:float):
        assert len(self.NM_list) == len(self.FT_list)
        self.NM_list.append(NM)
        self.FT_list.append(FT)

class PResult():
    def __init__(self):
        self.dict = {}
    
    def add_result(self, arg_dict:dict, NM:float, FT:float, main_key_str="mask_p"):
        # print(arg_dict)
        main_key = float(arg_dict[main_key_str])
        if not(main_key in self.dict.keys()):
            self.dict[main_key] = PResultItem(main_key)
        self.dict[main_key].add_result(NM, FT)

In [8]:
dir_path = "./results/eDiff"
file_blob = os.listdir(dir_path)
main_key_str = "mask_p"
main_key_str = "train_epoch"
results = PResult()
for fn in file_blob:
    cfn = os.path.join(dir_path, fn)
    with open(cfn) as f:
        f = f.read().split("\n")
    arg_dict, NM, FT = parse_file(f)
    results.add_result(arg_dict, NM, FT, main_key_str)

In [9]:
for j in np.sort(list(results.dict.keys())):
    i = results.dict[j]
    NM = np.mean(i.NM_list)
    FT = np.mean(i.FT_list)
    gain= FT - NM
    print(f"{main_key_str} = {i.main_key:.2f}, NM: {NM:.4f}, FT: {FT:.4f}, Gain: {gain:.4f}")

train_epoch = 50.00, NM: 0.9401, FT: 0.9414, Gain: 0.0013
train_epoch = 100.00, NM: 0.9378, FT: 0.9386, Gain: 0.0008
train_epoch = 150.00, NM: 0.9424, FT: 0.9427, Gain: 0.0002
train_epoch = 200.00, NM: 0.9424, FT: 0.9442, Gain: 0.0018


In [7]:
a = os.listdir("./results/traces")